# Creating a pipeline for chosen model

Basically running everyting again after evaluating different models, but this time in a more robust way to ensure a clean, reproducable pipeline.

For this, I got help from Google, other repos and ChatGPT to learn a little bit about SimpleImputer, OneHotEncoder and Pipeline.

In [20]:
from taxipred.utils.constants import CLEANED_DATA

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

## Going over all steps again, with some minor changes.

In [21]:
df = pd.read_csv(CLEANED_DATA/"taxi_prices_cleaned.csv")

In [22]:
# Splitting features and target

X, y = df.drop(columns="trip_price"), df["trip_price"]

In [23]:
# Train | Test-split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [24]:
# Picking numerical and categorical features

num_features = X_train.select_dtypes(include="number").columns
cat_features = X_train.select_dtypes(include="object").columns

In [ ]:
# Deciding which stragegy in "dummy encoding" the two different kinds of features should have

num_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore", drop="first"))])

num_transformer, cat_transformer

(Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
 Pipeline(steps=[('onehot',
                  OneHotEncoder(drop='first', handle_unknown='ignore'))]))

In [29]:
# Using ColumnTransformer to apply different preprocessing steps
# to numerical and categorical features in a single, clean preprocessing stage.

preprocess = ColumnTransformer(
    transformers=[("num", num_transformer, num_features), ("cat", cat_transformer, cat_features)]
)
preprocess

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [27]:
# Saving chosen model in a variable

model = RandomForestRegressor(random_state=42)

### Create full pipeline

Combining preprocessing and model into a single pipeline, so that all steps are consistent during training and prediction.

In [28]:
rf_pipeline = Pipeline(steps=[("preprocess", preprocess), "model", model])